# Get derived data and merge data sets

In [7]:
import pandas as pd
import geopandas as gpd
import numpy as np
from fuzzywuzzy import process, fuzz
import os

In [ ]:
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
DS_dir = currentdir + '/Data_Source'

try:
    os.mkdir(DS_dir, mode = 0o666)

except FileExistsError:
    
    pass

## Get Crime Rate(%)

In [2]:
def getCrimeRate(crime_police, ocean):
    crime_police['dummy'] = True
    ocean_block = ocean[['NAME','POP2010']].copy()
    ocean_block[['dummy']] = True
    
    pre_crime_police = pd.merge(crime_police, ocean_block, on = 'dummy')
    pre_crime_police.drop('dummy', axis = 1, inplace = True)

    pre_crime_police['Token_Set_Ratio'] = pre_crime_police[['P_name', 'NAME']].apply(lambda x:fuzz.token_set_ratio(x.P_name, x.NAME), axis = 1)
    pre_crime_police['Rank_Token_Set_Ratio'] = pre_crime_police.groupby('P_name')['Token_Set_Ratio'].rank(ascending = False, method = 'dense')
   
    pre_crime_police = pre_crime_police.loc[(pre_crime_police.Rank_Token_Set_Ratio == 1) & (pre_crime_police.Token_Set_Ratio > 60)]
    pre_crime_police.insert(19, 'crime_rate(%)', pre_crime_police['case_number']/pre_crime_police['POP2010']*20)
    pre_crime_police = pre_crime_police.iloc[:,:-2]
    
    crime_police = pd.merge(ocean_county, pre_crime_police,  how='right', left_on = ['NAME','POP2010'], right_on = ['NAME','POP2010'])
    
    return crime_police

In [5]:
MB_nj = gpd.read_file('./Maps/map_02/Municipal_Boundaries_of_NJ.shp')

ocean_county = MB_nj[MB_nj['COUNTY'] == 'OCEAN']
ocean_county = ocean_county.to_crs(epsg = 4326)

In [8]:
crime_police = pd.read_csv('./Data_Source/crime_police.csv')
crime_rate = getCrimeRate(crime_police, ocean_county)
crime_rate

,OBJECTID,MUN,COUNTY,MUN_LABEL,MUN_TYPE,NAME,GNIS_NAME,GNIS,SSN,MUN_CODE,...,property-crime,rape,rape-legacy,robbery,violent-crime,case_number,crime_rate(%),P_zip,P_latitude,P_longitude
0,132,BERKELEY TWP,OCEAN,Berkeley Township,Township,Berkeley Township,Township of Berkeley,882073,1505,1506,...,2101,32,0,43,198,4409,2.137438,8721,39.905606,-74.234437
1,12,BEACH HAVEN BORO,OCEAN,Beach Haven Borough,Borough,Beach Haven Borough,Borough of Beach Haven,885152,1503,1504,...,527,1,0,0,6,1060,18.119658,8008,39.563049,-74.241287
2,182,PLUMSTED TWP,OCEAN,Plumsted Township,Township,Plumsted Township,Township of Plumsted,882078,1523,1524,...,177,6,0,2,20,374,0.888256,8533,40.066606,-74.523211
3,166,MANTOLOKING BORO,OCEAN,Mantoloking Borough,Borough,Mantoloking Borough,Borough of Mantoloking,885290,1519,1520,...,43,0,0,0,0,86,5.810811,8738,40.036987,-74.052316
4,172,BAY HEAD BORO,OCEAN,Bay Head Borough,Borough,Bay Head Borough,Borough of Bay Head,885150,1502,1503,...,84,0,0,0,2,170,3.512397,8742,40.069550,-74.046565
5,119,OCEAN GATE BORO,OCEAN,Ocean Gate Borough,Borough,Ocean Gate Borough,Borough of Ocean Gate,885333,1521,1522,...,170,0,0,1,3,343,3.411238,8740,39.924434,-74.140403
6,49,STAFFORD TWP,OCEAN,Stafford Township,Township,Stafford Township,Township of Stafford,882069,1530,1531,...,1025,8,0,8,102,2156,1.625024,8050,39.691843,-74.250398
7,63,BARNEGAT TWP,OCEAN,Barnegat Township,Township,Barnegat Township,Township of Barnegat,882070,1533,1501,...,678,9,0,14,72,1434,1.369889,8005,39.758761,-74.244876
8,83,TUCKERTON BORO,OCEAN,Tuckerton Borough,Borough,Tuckerton Borough,Borough of Tuckerton,885422,1532,1533,...,190,0,0,1,10,390,2.330445,8087,39.592867,-74.338630
9,176,LAKEWOOD TWP,OCEAN,Lakewood Township,Township,Lakewood Township,Township of Lakewood,882076,1514,1515,...,4700,42,0,239,711,10138,2.183902,8701,40.093592,-74.213165


In [11]:
# crime_rate.to_csv('./Data_Source/crime_rate.csv', index = False)

In [9]:
crime_rate.columns

Index(['OBJECTID', 'MUN', 'COUNTY', 'MUN_LABEL', 'MUN_TYPE', 'NAME',
       'GNIS_NAME', 'GNIS', 'SSN', 'MUN_CODE', 'KEY', 'CENSUS2010', 'ACRES',
       'SQ_MILES', 'POP2010', 'POP2000', 'POP1990', 'POP1980', 'POPDEN2010',
       'POPDEN2000', 'POPDEN1990', 'POPDEN1980', 'GLOBALID', 'Shape_Leng',
       'Shape_Area', 'geometry', 'P_ID', 'P_name', 'P_address', 'P_city',
       'ori', 'agency_name', 'aggravated-assault', 'arson', 'burglary',
       'homicide', 'human-trafficing', 'larceny', 'motor-vehicle-theft',
       'property-crime', 'rape', 'rape-legacy', 'robbery', 'violent-crime',
       'case_number', 'crime_rate(%)', 'P_zip', 'P_latitude', 'P_longitude'],
      dtype='object')

In [10]:
cols = crime_rate.columns.tolist()
cols = ['P_ID', 'P_name', 'P_address', 'P_city', 'ori', 'agency_name', 'NAME'] + cols[-4:-3] + ['POPDEN2010'] + cols[-3:]
crime_police_ult = crime_rate[cols].copy()
crime_police_ult.rename(columns = {'NAME':'mun_name', 'POPDEN2010': 'POPDEN2010 (per sq. mi.)'}, inplace = True)
crime_police_ult

crime_police_ult.nlargest(35,'POPDEN2010 (per sq. mi.)')

,P_ID,P_name,P_address,P_city,ori,agency_name,mun_name,crime_rate(%),POPDEN2010 (per sq. mi.),P_zip,P_latitude,P_longitude
19,13860,POINT PLEASANT POLICE DEPARTMENT,2233 BRIDGE AVENUE,POINT PLEASANT,NJ0152400,POINT PLEASANT POLICE DEPARTMENT,Point Pleasant Borough,1.619182,4370,8742,40.072647,-74.069450
13,12479,BEACHWOOD POLICE DEPARTMENT,1600 PINEWALD ROAD,BEACHWOOD,NJ0150400,BEACHWOOD POLICE DEPARTMENT,Beachwood Borough,2.234495,3983,8722,39.917766,-74.207233
23,14103,SEASIDE HEIGHTS POLICE DEPARTMENT,116 SHERMAN AVENUE,SEASIDE HEIGHTS,NJ0152600,SEASIDE HEIGHTS POLICE DEPARTMENT,Seaside Heights Borough,12.836855,3891,8751,39.943737,-74.073740
5,8276,OCEAN GATE BOROUGH POLICE DEPARTMENT,801 OCEAN GATE AVENUE,OCEAN GATE,NJ0152100,OCEAN GATE POLICE DEPARTMENT,Ocean Gate Borough,3.411238,3706,8740,39.924434,-74.140403
9,10336,LAKEWOOD POLICE DEPARTMENT,231 3RD STREET,LAKEWOOD,NJ0151400,LAKEWOOD TOWNSHIP POLICE DEPARTMENT,Lakewood Township,2.183902,3700,8701,40.093592,-74.213165
21,14069,BOROUGH OF PINE BEACH POLICE DEPARTMENT,599 PENNSYLVANIA AVENUE,PINE BEACH,NJ0152200,PINE BEACH POLICE DEPARTMENT,Pine Beach Borough,1.673719,3269,8741,39.933579,-74.169849
16,12585,SOUTH TOMS RIVER POLICE DEPARTMENT,144 MILL STREET,SOUTH TOMS RIVER,NJ0152900,SOUTH TOMS RIVER POLICE DEPARTMENT,South Toms River Borough,3.441911,2986,8757,39.941755,-74.202643
25,19277,LAKEHURST POLICE DEPARTMENT,5 UNION AVENUE,LAKEHURST,NJ0151300,LAKEHURST POLICE DEPARTMENT,Lakehurst Borough,2.132630,2690,8733,40.012571,-74.309923
30,21138,POINT PLEASANT BEACH POLICE DEPARTMENT,416 NEW JERSEY AVENUE,POINT PLEASANT BEACH,NJ0152500,POINT PLEASANT BEACH POLICE DEPARTMENT,Point Pleasant Beach Borough,3.884244,2496,8742,40.086954,-74.046908
15,12489,BRICK TOWNSHIP POLICE DEPARTMENT,401 CHAMBERS BRIDGE ROAD,BRICK,NJ0150600,BRICK TOWNSHIP POLICE DEPARTMENT,Brick Township,2.643063,2316,8723,40.075261,-74.152465


In [ ]:
#crime_police_ult.to_csv('./Data_Source/crime_police_ult.csv', index = False)

## Merge data sets to get Final.csv

In [5]:
home = pd.read_csv('./Data_Source/home_ult.csv')
school = pd.read_csv('./Data_Source/school.csv')
fire = pd.read_csv('./Data_Source/fire.csv')
police = pd.read_csv('./Data_Source/crime_police_ult.csv')

In [6]:
home

,property_id,price,beds,baths,size(sqft),line,city,state,zipcode,longitude,latitude,days_ago,features
0,M6602530331,18500,2,2,1000,525 Main C St,Tuckerton,NJ,8087,-74.322433,39.615565,25,single_story
1,M9930412229,29900,2,2,784,469 S Green St Trlr 27,Tuckerton,NJ,8087,-74.338429,39.590329,78,single_story
2,M9471690149,35000,1,1,782,19D Monticello Dr,Whiting,NJ,8759,-74.369100,39.956901,12,single_story
3,M5847512768,38500,1,1,537,10G Bennington Ln Unit G,Whiting,NJ,8759,-74.369306,39.959331,25,single_story
4,M9269193256,39000,2,1,850,B Molly Pitcher Blvd Unit 11,Whiting,NJ,8759,-74.375238,39.960069,9,single_story
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2229,M9886104467,4500000,6,5,4430,14 Cummins St,Mantoloking,NJ,8738,-74.057772,40.006954,25,"garage_1_or_more, garage_2_or_more, two_or_mor..."
2230,M9479483724,5700000,7,8,4480,1324 Ocean Front,Point Pleasant Beach,NJ,8742,-74.039428,40.083495,19,"basement, garage_1_or_more, garage_2_or_more, ..."
2231,M5334622826,6500000,8,8,7219,91 Pershing Blvd,Lavallette,NJ,8735,-74.083205,39.965606,139,"garage_1_or_more, garage_2_or_more, two_or_mor..."
2232,M5495064600,9250000,7,8,7700,1211 Ocean Ave,Mantoloking,NJ,8738,-74.049089,40.038775,84,"basement, garage_1_or_more, garage_2_or_more, ..."


In [7]:
school

,S_ID,S_name,S_address,S_level,S_city,S_zip,S_latitude,S_longitude
0,264,FREDERIC A. PRIFF ELEMENTARY SCHOOL,139 WELLS MILL ROAD,MIDDLE,WARETOWN,8758,39.790614,-74.201966
1,2741,CENTRAL REGIONAL MIDDLE SCHOOL,509 FOREST HILLS PARKWAY,MIDDLE,BAYVILLE,8721,39.889370,-74.210436
2,2822,PINELANDS REGIONAL JUNIOR HIGH SCHOOL,590 NUGENTOWN ROAD,MIDDLE,LITTLE EGG HARBOR,8087,39.614155,-74.360452
3,3630,MIDSTREAMS ELEMENTARY SCHOOL,500 MIDSTREAMS RD,ELEMENTARY,BRICK TOWN,8724,40.068134,-74.100071
4,5809,OCEAN ACADEMY CHARTER SCHOOL,678 5TH ST,ELEMENTARY,LAKEWOOD,8701,40.093754,-74.223589
...,...,...,...,...,...,...,...,...
115,97998,OCEAN ACRES ELEMENTARY SCHOOL,489 NAUTILUS DRIVE,ELEMENTARY,MANAHAWKIN,8050,39.741927,-74.280700
116,100056,OCEAN CO JUV DET CTR,165 SUNSET AVENUE,HIGH,TOMS RIVER,8754,40.005922,-74.226227
117,100319,BARNEGAT HIGH SCHOOL,180 BENGAL BLVD,HIGH,BARNEGAT,8005,39.771832,-74.234156
118,100431,NEW EGYPT MIDDLE SCHOOL,115 EVERGREEN ROAD,MIDDLE,NEW EGYPT,8533,40.079741,-74.528927


In [8]:
fire

,F_ID,F_name,F_address,F_city,F_zip,F_latitude,F_longitude
0,4781,Cassville Volunteer Fire Company Station 56,12 Toms River Road,Jackson,8527,40.104000,-74.386833
1,5177,Jackson Mills Volunteer Fire Company Station 2,550 West Commodore Boulevard,Jackson,8527,40.156765,-74.389862
2,5593,Six Flags Great Adventure Safety Department,1 Six Flags Boulevard,Jackson,8527,40.141055,-74.445171
3,22074,Tuckerton Fire Department,111 North Green Street,Tuckerton,8087,39.603992,-74.339928
4,23145,Barnegat Light Volunteer Fire Company Station 13,907 Central Avenue,Barnegat Light,8006,39.756428,-74.107544
...,...,...,...,...,...,...,...
67,51148,Herbertsville Fire Company 1 Station 24,601 Herbertsville Road,Brick,8724,40.111195,-74.107506
68,51357,Lakewood Township Fire Department Station 66,976 New Hampshire Avenue,Lakewood,8701,40.053578,-74.196650
69,51358,Lakewood Township Fire Department Station 65,1350 Lanes Mill Road,Lakewood,8701,40.100643,-74.170769
70,51359,Manitou Park Volunteer Fire Company Station 18,110 3rd Avenue,Toms River,8757,39.945943,-74.215366


In [9]:
police

,P_ID,P_name,P_address,P_city,ori,agency_name,mun_name,crime_rate(%),POPDEN2010 (per sq. mi.),P_zip,P_latitude,P_longitude
0,5443,BERKELEY TOWNSHIP POLICE DEPARTMENT,631 PINEWALD KESWICK ROAD,BAYVILLE,NJ0150500,BERKELEY TOWNSHIP POLICE DEPARTMENT,Berkeley Township,2.137438,762,8721,39.905606,-74.234437
1,6221,BEACH HAVEN POLICE DEPARTMENT,300 ENGLESIDE AVENUE,BEACH HAVEN,NJ0150300,BEACH HAVEN POLICE DEPARTMENT,Beach Haven Borough,18.119658,502,8008,39.563049,-74.241287
2,6514,PLUMSTED TOWNSHIP POLICE DEPARTMENT,2 CEDAR STREET,NEW EGYPT,NJ0152300,PLUMSTED TOWNSHIP POLICE DEPARTMENT,Plumsted Township,0.888256,213,8533,40.066606,-74.523211
3,7382,MANTOLOKING POLICE DEPARTMENT,202 DOWNER AVENUE,MANTOLOKING,NJ0151900,MANTOLOKING POLICE DEPARTMENT,Mantoloking Borough,5.810811,465,8738,40.036987,-74.052316
4,7389,BAY HEAD POLICE DEPARTMENT,81 BRIDGE AVENUE,BAY HEAD,NJ0150200,BAY HEAD POLICE DEPARTMENT,Bay Head Borough,3.512397,1299,8742,40.069550,-74.046565
5,8276,OCEAN GATE BOROUGH POLICE DEPARTMENT,801 OCEAN GATE AVENUE,OCEAN GATE,NJ0152100,OCEAN GATE POLICE DEPARTMENT,Ocean Gate Borough,3.411238,3706,8740,39.924434,-74.140403
6,9165,STAFFORD TOWNSHIP POLICE DEPARTMENT,260 EAST BAY AVENUE,MANAHAWKIN,NJ0153000,STAFFORD TOWNSHIP POLICE DEPARTMENT,Stafford Township,1.625024,485,8050,39.691843,-74.250398
7,9168,BARNEGAT TOWNSHIP POLICE DEPARTMENT,900 WEST BAY AVENUE,BARNEGAT,NJ0153300,BARNEGAT TOWNSHIP POLICE DEPARTMENT,Barnegat Township,1.369889,520,8005,39.758761,-74.244876
8,10168,TUCKERTON BOROUGH POLICE DEPARTMENT,445 SOUTH GREEN STREET,TUCKERTON,NJ0153200,TUCKERTON POLICE DEPARTMENT,Tuckerton Borough,2.330445,909,8087,39.592867,-74.338630
9,10336,LAKEWOOD POLICE DEPARTMENT,231 3RD STREET,LAKEWOOD,NJ0151400,LAKEWOOD TOWNSHIP POLICE DEPARTMENT,Lakewood Township,2.183902,3700,8701,40.093592,-74.213165


In [22]:
def getClosetFacilities(Home, facilities):
    
    DF = []
    DF_all = []
    
    for f in facilities:
        
        facility = pd.DataFrame(f)
    
        Match = Home.merge(facility.iloc[:,-3:], how = "left", left_on = 'zipcode', right_on = facility.columns[-3][0:2] + 'zip')
        Match['dist_' + Match.columns[-1][0:1] + ' (mi.)'] = (((Match['longitude'] - 
                                                                 Match[Match.columns[-2][0:2] + 'longitude'])*54.6)**2 +
                                                                ((Match['latitude'] - 
                                                                 Match[Match.columns[-2][0:2] + 'latitude'])*69)**2)**0.5

        Same_zip_dist = Match[Match.iloc[:,-1].notnull()].groupby('property_id').agg({Match.columns[-1]:min}).reset_index()
        Match_same_zip = Match.merge(Same_zip_dist, how = 'right')

        _null = Match[Match.iloc[:,-1].isnull()].iloc[:,:13].reset_index(drop = True)
        Crs_diff_zip = _null.merge(facility.iloc[:,-3:], how = 'cross')
        Crs_diff_zip['dist_' + Match.columns[-2][0:1] + ' (mi.)'] = (((Crs_diff_zip['longitude'] - 
                                                                        Crs_diff_zip[Match.columns[-2][0:2] + 'longitude'])*54.6)**2 +
                                                                       ((Crs_diff_zip['latitude'] -
                                                                        Crs_diff_zip[Match.columns[-2][0:2] + 'latitude'])*69)**2)**0.5

        Diff_zip_dist = Crs_diff_zip.groupby('property_id').agg({Match.columns[-1]:min}).reset_index()
        Match_diff_zip = Crs_diff_zip.merge(Diff_zip_dist, how = 'right')
        
        
        DF = pd.concat([Match_same_zip, Match_diff_zip]).reset_index(drop = True)
        
        if len(DF_all) == 0:
            DF_all = DF
        
        else:
            DF_all = DF_all.merge(DF)

    return DF_all

In [23]:
Facilities = [police, school, fire]

In [24]:
near_all = getClosetFacilities(home, Facilities)
near_all

,property_id,price,beds,baths,size(sqft),line,city,state,zipcode,longitude,...,P_longitude,dist_P (mi.),S_zip,S_latitude,S_longitude,dist_S (mi.),F_zip,F_latitude,F_longitude,dist_F (mi.)
0,M5001559427,410000,4,2,1440,104 Emily Dr,Manahawkin,NJ,8050,-74.242958,...,-74.250398,0.964373,8050.0,39.702271,-74.256461,1.756368,8050.0,39.694973,-74.253075,1.222527
1,M5001666334,4299999,4,5,4222,1050 Barnegat Ln,Mantoloking,NJ,8738,-74.049687,...,-74.052316,0.741568,8742.0,40.069612,-74.050182,1.523856,8738.0,40.037197,-74.051582,0.720514
2,M5002680555,399000,4,3,1920,86 Susan Dr,Jackson,NJ,8527,-74.275922,...,-74.354088,4.715903,8527.0,40.148938,-74.267063,0.833662,8527.0,40.148136,-74.268967,0.730220
3,M5003192128,749000,3,2,1812,152 Evelyn Dr,Manahawkin,NJ,8050,-74.240649,...,-74.250398,1.465011,8050.0,39.702271,-74.256461,2.256102,8050.0,39.694973,-74.253075,1.720297
4,M5003905256,219900,2,2,1514,24 Jericho Rd,Toms River,NJ,8757,-74.283706,...,-74.202643,4.647112,8757.0,39.940520,-74.211638,4.211640,8757.0,39.965088,-74.245430,2.098825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2229,M9949368587,764000,5,4,3428,1675 Feldmus Ln Unit 3,Toms River,NJ,8755,-74.208461,...,-74.184588,2.170604,8755.0,40.027567,-74.225452,1.394424,8755.0,40.002415,-74.216492,0.821281
2230,M9958978876,209999,1,2,888,Court B Unit 1201,Brick,NJ,8724,-74.143564,...,-74.152465,0.578373,8724.0,40.083268,-74.118349,1.397334,8724.0,40.070724,-74.132332,0.876761
2231,M9969507043,695000,4,4,2980,1675 Feldmus Ln Unit 1,Toms River,NJ,8755,-74.208660,...,-74.184588,2.180283,8755.0,40.027567,-74.225452,1.384270,8755.0,40.002415,-74.216492,0.818884
2232,M9975791063,724900,4,4,2900,113 Kristjan Ct,Toms River,NJ,8755,-74.216805,...,-74.184588,1.999465,8755.0,40.027567,-74.225452,1.886189,8755.0,40.002415,-74.216492,0.092246


In [25]:
Final = near_all.merge(police.iloc[:,-5:])
Final

,property_id,price,beds,baths,size(sqft),line,city,state,zipcode,longitude,...,S_zip,S_latitude,S_longitude,dist_S (mi.),F_zip,F_latitude,F_longitude,dist_F (mi.),crime_rate(%),POPDEN2010 (per sq. mi.)
0,M5001559427,410000,4,2,1440,104 Emily Dr,Manahawkin,NJ,8050,-74.242958,...,8050.0,39.702271,-74.256461,1.756368,8050.0,39.694973,-74.253075,1.222527,1.625024,485
1,M5003192128,749000,3,2,1812,152 Evelyn Dr,Manahawkin,NJ,8050,-74.240649,...,8050.0,39.702271,-74.256461,2.256102,8050.0,39.694973,-74.253075,1.720297,1.625024,485
2,M5018130177,484900,4,2,1786,1389 E Mallard Dr,Manahawkin,NJ,8050,-74.214566,...,8050.0,39.702271,-74.256461,3.034089,8050.0,39.694973,-74.253075,2.576884,1.625024,485
3,M5022367641,339000,3,2,1378,1062 Driftwood Ave,Manahawkin,NJ,8050,-74.275864,...,8050.0,39.741927,-74.280700,0.726137,8050.0,39.734573,-74.279594,0.264668,1.625024,485
4,M5024752662,395000,3,2,992,1219 Coast Ave,Manahawkin,NJ,8050,-74.286465,...,8050.0,39.741927,-74.280700,0.837592,8050.0,39.734573,-74.279594,0.461485,1.625024,485
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2229,M6303112181,3399000,5,5,4966,14 E 29th St,Barnegat Light,NJ,8006,-74.114924,...,8758.0,39.788745,-74.201119,5.759499,8006.0,39.756428,-74.107544,1.162269,8.842730,254
2230,M6597277240,1499000,4,4,2462,38 W 8th St,Barnegat Light,NJ,8006,-74.109892,...,8758.0,39.788745,-74.201119,5.395256,8006.0,39.756428,-74.107544,0.202421,8.842730,254
2231,M6735381003,895000,3,3,1506,16 W 6th St,Barnegat Light,NJ,8006,-74.108170,...,8758.0,39.788745,-74.201119,5.449364,8006.0,39.756428,-74.107544,0.247408,8.842730,254
2232,M6921655107,1649000,4,3,2116,2202 Central Ave,Barnegat Light,NJ,8006,-74.112822,...,8758.0,39.788745,-74.201119,5.641933,8006.0,39.756428,-74.107544,0.757755,8.842730,254


In [26]:
Final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2234 entries, 0 to 2233
Data columns (total 27 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   property_id               2234 non-null   object 
 1   price                     2234 non-null   int64  
 2   beds                      2234 non-null   int64  
 3   baths                     2234 non-null   int64  
 4   size(sqft)                2234 non-null   int64  
 5   line                      2234 non-null   object 
 6   city                      2234 non-null   object 
 7   state                     2234 non-null   object 
 8   zipcode                   2234 non-null   int64  
 9   longitude                 2234 non-null   float64
 10  latitude                  2234 non-null   float64
 11  days_ago                  2234 non-null   int64  
 12  features                  2234 non-null   object 
 13  P_zip                     2234 non-null   float64
 14  P_latitu

In [27]:
#Final.to_csv('./Data_Source/Final.csv', index = False)